In [1]:
import typing as tp
from pathlib import Path
from functools import partial
from dataclasses import dataclass, field

import pandas as pd
import pyctcdecode
import numpy as np
from tqdm.notebook import tqdm

import librosa

import pyctcdecode
import kenlm
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2ProcessorWithLM, Wav2Vec2ForCTC
from bnunicodenormalizer import Normalizer

/home/zhenlanwang0426/.venv311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
SAMPLING_RATE = 16_000

### load model, processor, decoder

In [3]:
model = torch.load('model/best.pth',map_location=torch.device('cpu'))

In [4]:
processor = Wav2Vec2Processor.from_pretrained('model')
vocab_dict = processor.tokenizer.get_vocab()
sorted_vocab_dict = {k: v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}

## prepare dataloader

In [5]:
class BengaliSRTestDataset(torch.utils.data.Dataset):
    
    def __init__(
        self,
        audio_paths: list[str],
        sampling_rate: int
    ):
        self.audio_paths = audio_paths
        self.sampling_rate = sampling_rate
        
    def __len__(self,):
        return len(self.audio_paths)
    
    def __getitem__(self, index: int):
        audio_path = self.audio_paths[index]
        sr = self.sampling_rate
        w = librosa.load(audio_path, sr=sr, mono=False)[0]
        
        return w

In [6]:
test = pd.read_csv("data/train.csv").iloc[:1024]
test

,id,sentence,split
0,000005f3362c,ও বলেছে আপনার ঠিকানা!,train
1,00001dddd002,কোন মহান রাষ্ট্রের নাগরিক হতে চাও?,train
2,00001e0bc131,"আমি তোমার কষ্টটা বুঝছি, কিন্তু এটা সঠিক পথ না।",train
3,000024b3d810,নাচ শেষ হওয়ার পর সকলে শরীর ধুয়ে একসঙ্গে ভোজন...,train
4,000028220ab3,"হুমম, ওহ হেই, দেখো।",train
...,...,...,...
1019,00417912a6ee,আমাদের সঙ্গে ছিলেন এক বৃদ্ধ সাধু।,train
1020,0041949399ee,"প্রেম, মার তাকে!",train
1021,0041a6298d5c,আমি তাকে অবাধ্য হতে প্ররোচিত করিনি।,train
1022,0041a78a26ec,তিনি বর্তমানে হেভিওয়েট বিভাগে প্রতিদ্বন্দ্বিত...,train


In [7]:
test_audio_paths = [ "data/train_mp3s/"+f"{aid}.mp3" for aid in test["id"].values]

In [8]:
test_dataset = BengaliSRTestDataset(
    test_audio_paths, SAMPLING_RATE
)

collate_func = partial(
    processor.feature_extractor,
    return_tensors="pt", sampling_rate=SAMPLING_RATE,
    padding=True,
)

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=32, shuffle=False,
    num_workers=8, collate_fn=collate_func, drop_last=False,
    pin_memory=True,
)

## Inference

In [9]:
if not torch.cuda.is_available():
    device = torch.device("cpu")
else:
    device = torch.device("cuda")
print(device)

cpu


In [10]:
model = model.to(device)
model = model.eval()

In [11]:
pred_sentence_list = []
with torch.no_grad():
    for batch in test_loader:
        x = batch["input_values"]
        y = model(x).logits
        y = y.detach().cpu().numpy()
        pred_sentence_list.append(y)

2023-09-03 18:56:06.618382: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-09-03 18:56:06.632117: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-09-03 18:56:06.637302: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-09-03 18:56:06.637808: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-09-03 18:56:06.659132: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-09-03 18:56:06.672804: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-09-03 18:56:06.726244: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-09-03 18:56:07.000831: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
tcmalloc

In [18]:
import evaluate
metric = evaluate.load("wer")

2023-09-03 20:04:14.356756: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [32]:
def WER(args):
    alpha,beta = args[0],args[1]
    decoder = pyctcdecode.build_ctcdecoder(
    list(sorted_vocab_dict.keys()),
    'arijitx-wav2vec2-xls-r-300m-bengali/language_model/5gram.bin',
    alpha=alpha,beta=beta,
    )

    processor_with_lm = Wav2Vec2ProcessorWithLM(
        feature_extractor=processor.feature_extractor,
        tokenizer=processor.tokenizer,
        decoder=decoder
    )

    transcriptions = []
    for logits in pred_sentence_list:
        for logit in logits:
            transcriptions.append(processor_with_lm.decode(logit, beam_width=512).text)

    wer = metric.compute(predictions=transcriptions, references=test.sentence.values.tolist())
    return wer

In [31]:
import itertools

36

In [33]:
import multiprocessing
pool = multiprocessing.Pool(processes=36)

In [ ]:
scores = pool.map(WER, itertools.product(np.linspace(0.5,5,6), np.linspace(0.5,6,6)))

In [ ]:
with open(r"scores.pickle", "wb") as output_file:
    pickle.dump(scores, output_file)